# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(override=True) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [6]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [7]:
chain = prompt | model | output_parser

In [8]:
chain.invoke({"topic": "bears"})

"Why did the bear break up with the honey pot? \n\nBecause he couldn't bear the clingy nature of their relationship!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [9]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [10]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

/Users/ravikumartummalapenta/Documents/learning/llm/courses/funtions_tools/.function_tools/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
retriever.get_relevant_documents("where did harrison work?")

/var/folders/wj/kn8jgh51669650nr3m4hlnhc0000gn/T/ipykernel_12738/3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [12]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [14]:
from langchain.schema.runnable import RunnableMap

In [15]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [16]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [17]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [18]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [19]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(model='gpt-4o-mini-2024-07-18',temperature=0).bind(functions=functions)

In [21]:
runnable = prompt | model

In [22]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 62, 'total_tokens': 78, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-eeaea917-abf5-4bce-aebe-009f4276ad4c-0', usage_metadata={'input_tokens': 62, 'output_tokens': 16, 'total_tokens': 78, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [23]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [24]:
model = model.bind(functions=functions)

In [25]:
runnable = prompt | model

In [26]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 95, 'total_tokens': 113, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-d0cf152e-e3c2-4ff6-bd75-daa92672b5c8-0', usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## Fallbacks

In [27]:
from langchain.llms import OpenAI
import json

**Note**: Due to the deprecation of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [28]:
simple_model = ChatOpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-4o-mini-2024-07-18"
)
simple_chain = simple_model | json.loads

In [29]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [30]:
simple_model.invoke(challenge)

AIMessage(content='Here is a JSON blob containing three poems, each represented as a JSON object with a title, author, and first line:\n\n```json\n{\n  "poems": [\n    {\n      "title": "Whispers of the Dawn",\n      "author": "Emily Carter",\n      "first_line": "In the hush of morning light, dreams take flight."\n    },\n    {\n      "title": "The Dance of Leaves",\n      "author": "James Holloway",\n      "first_line": "Beneath the ancient oak, the leaves begin to sway."\n    },\n    {\n      "title": "Echoes of the Sea",\n      "author": "Sophia Lin",\n      "first_line": "Waves crash softly, singing tales of distant shores."\n    }\n  ]\n}\n```\n\nFeel free to modify any part of the JSON structure or the content as needed!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 179, 'prompt_tokens': 31, 'total_tokens': 210, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_pr

**Note**: The next line is expected to fail.

In [31]:
simple_chain.invoke(challenge)

TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

In [32]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [33]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [34]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose is a rose is a rose'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

In [35]:
final_chain = simple_chain.with_fallbacks([chain])

In [36]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose is a rose is a rose'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

## Interface

In [37]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [38]:
chain.invoke({"topic": "bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!"

In [39]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend?\n\nBecause she couldn't bear his bad puns!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [40]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 do
 bears
 have
 hairy
 coats
?


F
ur
 protection
!



In [41]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear dissolve in water?\n\nBecause it was a polar bear!'